In [134]:
import os
import numpy as np
import pandas as pd

In [176]:
def classify_accuracy(data: pd.DataFrame, label='', acc_classifier=0.5) -> pd.DataFrame:
    condition = [
        data['acc' + label] == 1,
        data['sales'] == 0,
        data['acc' + label] < 1 - acc_classifier,
        data['acc' + label] > 1 + acc_classifier
    ]
    class_label = label + '_cnt'
    values = ['cover' + class_label, 'zero' + class_label, 'less' + class_label, 'over' + class_label]
    data['class' + label] = np.select(condlist=condition, choicelist=values, default=None)
    data['class' + label] = data['class' + label].fillna('cover' + class_label)

    return data

def count_class(data: pd.DataFrame, grp_col: list, label=''):
    class_cnt = data.groupby(by=grp_col)['item_cd'].count().astype(int).reset_index().rename(columns={'item_cd': 'class_cnt'})

    class_cnt = class_cnt.pivot(
        index='cust_grp_nm',
        columns='class',
        values='class_cnt'
    ).fillna(0)

    for class_kind in ['cover', 'less', 'over']:
        if class_kind + label + '_cnt' not in class_cnt.columns:
            class_cnt[class_kind + label + '_cnt'] = 0
    
    class_cnt['tot_cnt'] = class_cnt.sum(axis=1)
    class_rate = class_cnt.div(class_cnt['tot_cnt'], axis=0)
    class_rate = class_rate.drop(columns=['tot_cnt'])
            
    return class_cnt, class_rate

In [177]:
data_path = os.path.join('..', 'data')
save_path = os.path.join('..', 'data')
global_w12 = 'fcst_global_w12.csv'
global_w13 = 'fcst_global_w13.csv'
sales_file = 'sales.csv'

In [178]:
gb_w12 = pd.read_csv(os.path.join(data_path, global_w12), delimiter='\t')
gb_w13 = pd.read_csv(os.path.join(data_path, global_w13), delimiter='\t')

In [179]:
gb_w12 = classify_accuracy(data=gb_w12)
gb_w12_count, gb_w12_rate = count_class(data=gb_w12, grp_col=['cust_grp_nm', 'class'])

gb_w13 = classify_accuracy(data=gb_w13)
gb_w13_count, gb_w13_rate = count_class(data=gb_w13, grp_col=['cust_grp_nm', 'class'])

In [180]:
gb_w12_rate

class,cover_cnt,less_cnt,over_cnt
cust_grp_nm,,,
Africa,0.894737,0.105263,0.000000
Central Asia,0.536585,0.000000,0.463415
China,0.560000,0.120000,0.320000
Europe,0.576923,0.269231,0.153846
Hong Kong,0.263158,0.157895,0.578947
Japan,1.000000,0.000000,0.000000
Middle East,0.357143,0.214286,0.428571
Mongolia,0.727273,0.136364,0.136364
North America,0.444444,0.222222,0.333333


In [181]:
gb_w13_rate

class,cover_cnt,less_cnt,over_cnt
cust_grp_nm,,,
Central Asia,0.928571,0.000000,0.071429
China,0.348837,0.069767,0.581395
Europe,0.347826,0.086957,0.565217
Hong Kong,0.379310,0.172414,0.448276
Japan,0.000000,0.428571,0.571429
Middle East,0.666667,0.000000,0.333333
Mongolia,0.357143,0.428571,0.214286
North America,0.300000,0.225000,0.475000
Oceania,1.000000,0.000000,0.000000


In [182]:
gb_w13_rate - gb_w12_rate

class,cover_cnt,less_cnt,over_cnt
cust_grp_nm,,,
Africa,NaN,NaN,NaN
Central Asia,0.391986,0.000000,-0.391986
China,-0.211163,-0.050233,0.261395
Europe,-0.229097,-0.182274,0.411371
Hong Kong,0.116152,0.014519,-0.130672
Japan,-1.000000,0.428571,0.571429
Middle East,0.309524,-0.214286,-0.095238
Mongolia,-0.370130,0.292208,0.077922
North America,-0.144444,0.002778,0.141667


In [158]:
gb_w12_count

class,cover_cnt,less_cnt,over_cnt,tot_cnt
cust_grp_nm,,,,
Africa,17.0,2.0,0.0,19.0
Central Asia,22.0,0.0,19.0,41.0
China,14.0,3.0,8.0,25.0
Europe,15.0,7.0,4.0,26.0
Hong Kong,10.0,6.0,22.0,38.0
Japan,1.0,0.0,0.0,1.0
Middle East,10.0,6.0,12.0,28.0
Mongolia,16.0,3.0,3.0,22.0
North America,4.0,2.0,3.0,9.0


In [159]:
gb_w13_count

class,cover_cnt,less_cnt,over_cnt,tot_cnt
cust_grp_nm,,,,
Central Asia,13.0,0.0,1.0,14.0
China,15.0,3.0,25.0,43.0
Europe,8.0,2.0,13.0,23.0
Hong Kong,11.0,5.0,13.0,29.0
Japan,0.0,3.0,4.0,7.0
Middle East,2.0,0.0,1.0,3.0
Mongolia,5.0,6.0,3.0,14.0
North America,12.0,9.0,19.0,40.0
Oceania,7.0,0.0,0.0,7.0


In [157]:
gb_w13_count - gb_w12_count

class,cover_cnt,less_cnt,over_cnt,tot_cnt
cust_grp_nm,,,,
Africa,NaN,NaN,NaN,NaN
Central Asia,-9.0,0.0,-18.0,-27.0
China,1.0,0.0,17.0,18.0
Europe,-7.0,-5.0,9.0,-3.0
Hong Kong,1.0,-1.0,-9.0,-9.0
Japan,-1.0,3.0,4.0,6.0
Middle East,-8.0,-6.0,-11.0,-25.0
Mongolia,-11.0,3.0,0.0,-8.0
North America,8.0,7.0,16.0,31.0


### ---

In [172]:
sales = pd.read_csv(os.path.join(data_path, sales_file))
sales.columns = [col.lower() for col in sales.columns]

In [173]:
check_cust_list = ['China', 'Europe', 'Mongolia', 'SouthWest Asia', 'Taiwan']

In [174]:
sales_filter = sales[sales['cust_grp_nm'].isin(check_cust_list)].copy()

In [175]:
sales_filter.to_csv(os.path.join(save_path, 'sales_global_problem.csv'), index=False)

In [197]:
sales_filter_comp = sales_filter[sales_filter['yymmdd'] == 2022].copy()
sales_filter_comp = sales_filter_comp[sales_filter_comp['week'].isin(['W12', 'W13'])].copy()

In [198]:
sales_filter_comp_sum = sales_filter_comp.groupby(by=['cust_grp_nm', 'yymmdd', 'week']).sum()['sales'].reset_index()
sales_filter_comp_sum

,cust_grp_nm,yymmdd,week,sales
0,China,2022,W12,11261.0
1,China,2022,W13,3530.0
2,Europe,2022,W12,640.0
3,Europe,2022,W13,580.0
4,Mongolia,2022,W12,2300.0
5,Mongolia,2022,W13,11270.0
6,SouthWest Asia,2022,W12,70.0
7,SouthWest Asia,2022,W13,283.0
8,Taiwan,2022,W12,10871.0
9,Taiwan,2022,W13,7912.0


In [199]:
sales_filter_comp_sum_w12 = sales_filter_comp_sum[sales_filter_comp_sum['week'] == 'W12']
sales_filter_comp_sum_w13 = sales_filter_comp_sum[sales_filter_comp_sum['week'] == 'W13']

In [208]:
sales_w12= sales_filter_comp_sum_w12['sales'].to_numpy() 
sales_w13 = sales_filter_comp_sum_w13['sales'].to_numpy() 

sales_rate = (sales_w13-sales_w12) * 100 / sales_w12

In [211]:
sales_rate

array([-68.65287275,  -9.375     , 390.        , 304.28571429,
       -27.21920706])